In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas import DataFrame, Series
import torch

In [6]:
train_stat = pd.read_csv('./filtered data/train_stat_feat_df.csv')
dev_stat = pd.read_csv('./filtered data/dev_stat_feat_df.csv')

dev_stat.head()

,tweet_id,reply_reply_count,reply_like_count,reply_retweet_count,reply_quote_count,reply_possibly_sensitive,reply_has_url,reply_mentioned_url_num,reply_id_num,reply_isweekday,...,quote_count,possibly_sensitive,has_url,mentioned_url_num,id_num,isweekday,followers_count,tweet_count,verified,senti_score
0,1249004694950817796,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.004405,0.000000,0.001066,0.000000,...,0.0,0.0,1.0,0.333333,0.000000,0.0,0.000557,0.051326,0.0,1.0
1,1267552274819227649,0.000048,0.000000e+00,0.000000,0.000000,0.0,0.008811,0.000000,0.004264,0.012048,...,0.0,0.0,1.0,0.000000,0.020408,1.0,0.000012,0.011393,0.0,0.0
2,1235238334722699265,0.000215,1.959079e-06,0.000007,0.000024,0.0,0.013216,0.000000,0.000000,0.060241,...,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000048,0.088659,0.0,0.0
3,1248746792914546688,0.000000,6.530263e-07,0.000000,0.000000,0.0,0.004405,0.000000,0.001066,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000015,0.013806,0.0,0.0
4,523820806917603328,0.000119,1.306053e-06,0.000004,0.000000,0.0,0.114537,0.054348,0.062900,0.024096,...,0.0,0.0,1.0,0.666667,0.020408,0.0,0.021542,0.037443,1.0,0.0


In [7]:
train_tweet = pd.read_csv('./filtered data/train_tweet_df.csv')
dev_tweet = pd.read_csv('./filtered data/dev_tweet_df.csv')

dev_tweet.head()

,tweet_id,text,created_at,user_id,tweet_id.1,label,reply,reply_text
0,1249004694950817796,covid fact hand dryer effect kill new coronavi...,2020-04-11T16:01:39.000Z,35761403,1249004694950817796,0,1249011200068730880,fact germ breed
1,1267552274819227649,expect result husband pend antibodi test pleas...,2020-06-01T20:23:06.000Z,2734457193,1267552274819227649,0,"1270394169836568576,1270502071175909376",hi luck boat
2,1235238334722699265,covid spread peopl catch covid other diseas sp...,2020-03-04T16:19:03.000Z,53980699,1235238334722699265,0,"1235234904281165825,1235234927937048577,123523...",read lot corona viru late think share highligh...
3,1248746792914546688,everi news outlet use headlin like antibiot ef...,2020-04-10T22:56:50.000Z,393187879,1248746792914546688,0,1248775858120097792,appar headlin question answer usual
4,523820806917603328,research encount goliath birdeat world largest...,2014-10-19T12:59:47.000Z,39585367,523820806917603328,0,"523821510042353664,523822210071658496,52382235...",eu tenho uma dessa em casa são ótima para coça...
